In [86]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [87]:
iOption0 = 0
test = 1
sim = 1

In [88]:
isigma = 2
N_files = 1
N_centr = 5
N_hists = 4
N_layers = 4
N_ilyaers = 8
is_sigmalised = 0
is_e = 0
iOption1 = 0
centralities = ["0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [1,2,3,4,5,6,7,8]
colors=[1,2,4,root.kGreen+2,root.kMagenta,3]
file_path="input/NewHadron/"
file_names="missalign_hadron_new1_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el","dthe_hist_el","sdphi_hist_el","sdthe_hist_el"]

In [89]:
if sim :
    file_path="../sim/"
    file_names="my-kek"

In [90]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if not test: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    if test: infile = root.TFile.Open(file_path+file_names+".root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

../sim/my-kek0.root


In [91]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.8,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.7, 0.9, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0, 1.4, 2.0, 2.8, 4.9])


In [92]:
boards_tot = [[-0.1,0.1],[-0.1,0.1]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
if iOption1==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]

In [93]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus{seed+p}', 'gaus',  boards_fit[int(seed0%2)][0], boards_fit[int(seed0%2)][1])
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
        Format_Func(g_gaus,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[int(seed0%2)][0], boards_tot[int(seed0%2)][1])
        Format_Func(total,2,6)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_gaus.SetParameter(1,sigma_ii)
            g_gaus.SetParameter(2,sigma_ii)
            g_gaus.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_gaus.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        else:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[int(seed0%2)][1])

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(seed0%2)][0]/10,boards_tot[int(seed0%2)][1]/10)
        total.SetParLimits(2,boards_fit[int(seed0%2)][1]/10,boards_tot[int(seed0%2)][1]/2)

        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1))
        Sigma_local_err.append(total.GetParError(2))

        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            

        m2_name=str(icentr)+"/"+str(ilayer)+"/emc_"+str(round(pt[p],1))

        c1.SaveAs( "output/delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [94]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [95]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3], i, isigma) for i in range(Ntr)])
pool.close()

In [96]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [97]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
colors=[1,2,4,6,root.kGreen+2]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)+x*[3]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-4.126,-0.023)
            fit_functions1[-1].SetParameter(3,0)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [98]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.03
    if iOption1==1: max, min = 4.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer%4],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

array('d', [0.01557727915655137, 0.006891446104359686, 0.01111731106754915, 0.005949482356522657, 0.006538207567243235, 0.005373310934362816, 0.004451666745116002, 0.003842240604697336, 0.003226404350406812, 0.0026949474447345812]) array('d', [0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012])


In [99]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}, {round(fits.GetParameter(3),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][5][4][3]={
{
{
{0.003682, 0.00783, -1.563218, -0.000261},
{-0.001952, 0.009431, -0.43468, 0.000697},
{0.00265, 0.009607, -1.478267, -9.7e-05},
{0.003383, 0.014241, -2.674573, -0.000314},
},
{
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
},
{
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
},
{
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
},
{
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
{0.01929, -0.008147, -0.398621, -0.001614},
},
},
};


In [100]:
colors=[1,2,4,6,root.kGreen+2]
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-3.126,-0.023)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [101]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",500*2,500*2)
    c3.Divide(2,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.003, 0.003
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer%4],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

In [102]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_file in fit_mean_functions:
    print("{")
    for fit_functions_syst in fit_functions_file:
        print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}"+"},")
    print("},")
print("};")

params[1][5][4][3]={
{
{5.3e-05, 9e-05,8.9e-05, 0.000283},
{-0.000169, -0.000169,-0.000169, -0.000169},
{-0.000169, -0.000169,-0.000169, -0.000169},
{-0.000169, -0.000169,-0.000169, -0.000169},
{-0.000169, -0.000169,-0.000169, -0.000169},
},
};
